In [3]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class SoftmaxRegression:
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Tránh tràn số
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Encode string labels to integer indices
        self.classes_, y_indices = np.unique(y, return_inverse=True)
        n_classes = len(self.classes_)
        
        # Khởi tạo trọng số và bias
        self.weights = np.zeros((n_features, n_classes))
        self.bias = np.zeros(n_classes)
        
        # Mã hóa one-hot cho nhãn
        y_one_hot = np.zeros((n_samples, n_classes))
        for i in range(n_samples):
            y_one_hot[i, y_indices[i]] = 1
        
        # Gradient Descent
        for _ in range(self.n_iters):
            # Tính xác suất Softmax
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.softmax(linear_model)
            
            # Tính gradient
            grad_w = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            grad_b = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)
            
            # Cập nhật tham số
            self.weights -= self.lr * grad_w
            self.bias -= self.lr * grad_b
            
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.softmax(linear_model)
        class_indices = np.argmax(y_pred, axis=1)
        # Convert indices back to original class labels
        return self.classes_[class_indices]

# Tải và tiền xử lý dữ liệu Iris
iris = pd.read_csv("input_2.csv")
X, y = iris.iloc[:, :-1].values, iris.iloc[:, -1].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Đọc dữ liệu test từ file csv, đảm bảo đọc đúng format
# Read first few lines to check if there's a header
with open("output_2.csv", "r") as f:
    first_line = f.readline().strip()
    # Check if the first line looks like a header or data
    if ',' in first_line and all(c.isdigit() or c in '.,+-' for c in first_line.replace(',', '')):
        # Looks like data, no header
        X_test = pd.read_csv("output_2.csv", header=None)
    else:
        # Has a header
        X_test = pd.read_csv("output_2.csv")

print(f"Number of test samples: {X_test.shape[0]}")
print(f"X_test data shape: {X_test.shape}")

# Huấn luyện mô hình
model = SoftmaxRegression(lr=0.1, n_iters=1000)
model.fit(X, y)

# Dự đoán trên tất cả mẫu test
# Chuẩn hóa dữ liệu test giống như dữ liệu train
X_test_scaled = scaler.transform(X_test.values)

# Dự đoán nhãn cho dữ liệu test
predictions = model.predict(X_test_scaled)
print(f"Number of predictions: {len(predictions)}")

# Ghi kết quả ra file
with open("iris_predictions.csv", "w") as f:
    f.write("Sample,Predicted_Label\n")
    for i, pred in enumerate(predictions):
        f.write(f"{i+1},{pred}\n")

print("Done writing predictions. Last sample number:", len(predictions))

Number of test samples: 30
X_test data shape: (30, 4)
Number of predictions: 30
Done writing predictions. Last sample number: 30
